![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/open-source-nlp/22.0.Llama2_Transformer_In_SparkNLP.ipynb)

# LLAMA2Transformer: CausalLM with Open Source models

> Llama 2 is a collection of pretrained and fine-tuned large language models (LLMs) ranging in scale from 7 billion to 70 billion parameters.

>[Source](https://ai.meta.com/research/publications/llama-2-open-foundation-and-fine-tuned-chat-models/)

LLAMA2Transfomer is compatible with quantized models (in INT4 or INT8) for CPUs, allowing the use of state-of-the-art models in consumer computers and environments. It supports ONNX exports and quantizations for:

* 16 bit (CUDA only)
* 8 bit (CPU or CUDA)
* 4 bit (CPU or CUDA)  

## Colab Setup

In [ ]:
!wget -q http://setup.johnsnowlabs.com/colab.sh -O - | bash

In [2]:
import sparknlp

from sparknlp.base import *
from sparknlp.annotator import *

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml import Pipeline, PipelineModel

**Make sure to Enable GPU Mode and High RAM**

In [3]:
spark = sparknlp.start()

# uncomment the next line to enable GPU mode
# spark = sparknlp.start(gpu=True)

print("Spark NLP version", sparknlp.version())
print("Apache Spark version:", spark.version)

spark

Spark NLP version 6.2.0
Apache Spark version: 3.5.1


## Llama2 Pipeline

Now, let's create a Spark NLP Pipeline with `llama_2_7b_chat_hf_int4` model and check the results.

In [4]:
document_assembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("documents")

llama2 = LLAMA2Transformer.pretrained()\
    .setMaxOutputLength(150)\
    .setDoSample(False)\
    .setInputCols(["documents"])\
    .setOutputCol("generation")

pipeline = Pipeline(
  stages=[
    document_assembler,
    llama2
])

llama_2_7b_chat_hf_int4 download started this may take some time.
Approximate size to download 3.5 GB
[OK!]


In [5]:
data = spark.createDataFrame([["Tell me a nice short history."]]).toDF("text")
result = pipeline.fit(data).transform(data)
result.select("generation.result").show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|result                                                                                                                                                                                                                                        

We can display the documentation of all params with their optionally default values and user-supplied values by `explainParams()` function

In [6]:
print(llama2.explainParams())

batchSize: Size of every batch (default: 1)
beamSize: Number of beams for beam search. (default: 1)
configProtoBytes: ConfigProto from tensorflow, serialized into byte array. Get with config_proto.SerializeToString() (undefined)
doSample: Whether or not to use sampling; use greedy decoding otherwise (default: False, current: False)
engine: Deep Learning engine used for this model (current: openvino)
ignoreTokenIds: A list of token ids which are ignored in the decoder's output (default: [])
inputCols: previous annotations columns, if renamed (current: ['documents'])
lazyAnnotator: Whether this AnnotatorModel acts as lazy in RecursivePipelines (default: False)
maxInputLength: Maximum length of the input sequence (default: 4096)
maxOutputLength: Maximum length of output text (default: 20, current: 150)
minOutputLength: Minimum length of the sequence to be generated (default: 0)
nReturnSequences: The number of sequences to return from the beam search. (undefined)
noRepeatNgramSize: If set 

Let's use model with more sentences and set `.setDoSample()` parameter as True, this parameter is used for whether or not to use sampling; use greedy decoding otherwise, by default False. <br/>
Also, we use `.setTopK()` parameter for the number of highest probability vocabulary tokens to keep for top-k-filtering, by default 50.

In [7]:
sample_texts= [[1, "Mey name is  Leonardo"],
               [2, "My name is Leonardo and I come from Rome."],
               [3, "My name is"],
               [4, "What is the difference between diesel and petrol?"]]

sample_df= spark.createDataFrame(sample_texts).toDF("id", "text")

In [8]:
llama2.setMaxOutputLength(50).setMinOutputLength(25).setDoSample(True).setTopK(20)

LLAMA2TRANSFORMER_a9d0be9dc3df

In [9]:
result = pipeline.fit(sample_df).transform(sample_df)
result.select("id", "generation.result").show(truncate=False)

+---+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|id |result                                                                                                                                                                                                                                                                         |
+---+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|1  |[_Mey name is  Leonardo. Unterscheidung between different types of neurons in the brain_\n\nNeurons are the basic building blocks of the nervous system, and ther

In [10]:
from typing import List


def prompts_to_spark_df(prompts,spark=spark):
  text = [[i, prompt] for i,prompt in enumerate(prompts)]
  return spark.createDataFrame(text).toDF("id", "text")


def generate_with_llm(llm_pipe, prompts, print=True):
  if isinstance(prompts,str):
    df = prompts_to_spark_df([prompts])
  elif isinstance(prompts,List):
    df = prompts_to_spark_df(prompts)
  else :
    raise ValueError(f"Invalid Type = {type(prompts)} please pass a str or list of str for prompts parameter ")
  df = llm_pipe.fit(df).transform(df)
  df = df.select("id",'text', "generation.result").toPandas()

  if print:
    print_generation_results(df)
  return df


def print_generation_results(df):
  for idx, row in df.iterrows():
    print(f'Example {idx}: {200*"_"}')
    print(row.result[0])
    print('\n')


# Explore Parameters Play with Paramns

### Sampling Methods


Sampling means we **randomly** draw from a distribution of words.
The probability distribution is conditioned on all previous tokens in a text to generate the next token.

By default the distribution contains all words in the vocabulary of GPT2, where many candidates are incorrect to generate.

There are two methods of reshaping and drawing from those distributions :

1. **Top-K Sampling** Take the k most likely words from the original distribution. Redistribute probability mass among those k words and draw according to the new probabilities.

2. **Top-P Nucleus sampling**  Take smallest possible set of N words, which  together have a probability of p. Redistribute probability mass among those N words and draw according to the new probabilities.



Additionally, both methods can be tweaked ith the following parameters :

- **temperature** : Parameter of the softmax function which affect the distrubtion computed by the model. The closer we are to 0, the more deterministic the probability will become, distribution tails will become slimmer and outlier word probabilites are more close to 0. Temperature values closer values to 1 make tails of probability fatter which makes outliers more probable and generic results less probable.


These parameters are shared by all method :
- **ignoreTokenIds**: A list of token ids which are ignored in the decoder's output (default: [])
- **noRepeatNgramSize**: If set to int > 0, all ngrams of that size can only occur once
- **repetitionPenalty**: The parameter for repetition penalty. 1.0 means no penalty.  https://arxiv.org/pdf/1909.05858.pdf>
- **task**:  Transformer's task, e.g. 'is it true that'> (default: , current: generate)

### Play with temperature
Set Temperature higher to make GPT more random/creative and text less coherent
Temperature > 0  and Temperature <=1
You must set `llama2.setDoSample(True)` to have non-deterministic results

In [11]:
text = """Hello my name is Llama, I love to """
data = [text, text,text,text,text ]
llama2.setMaxOutputLength(200)

LLAMA2TRANSFORMER_a9d0be9dc3df

In [12]:
llama2.setTemperature(1)
llama2.setDoSample(True)
generate_with_llm(pipeline, data, print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
_Hello my name is Llama, I love to  dance and I have a big smile on my face. Einzeln_

ð


Example 1: ________________________________________________________________________________________________________________________________________________________________________________________________________
_Hello my name is Llama, I love to 3D print and I'm not afraid to show off my creations on social media.("@Llama3D on Instagram, @LlamaThe3D printer on Twitter) 

I'm a big fan of the MakerBot Replicator+, it's the best 3D printer I've ever used. The print quality is amazing and the ease of use is unmatched. I've used it to create everything from functional prototypes to decorative objects. I'm always on the lookout for new and exciting projects to work on and I love to share my

,id,text,result
0,0,"Hello my name is Llama, I love to","[_Hello my name is Llama, I love to dance and..."
1,1,"Hello my name is Llama, I love to","[_Hello my name is Llama, I love to 3D print a..."
2,2,"Hello my name is Llama, I love to","[_Hello my name is Llama, I love to dance and..."
3,3,"Hello my name is Llama, I love to","[_Hello my name is Llama, I love to ❤️ dance a..."
4,4,"Hello my name is Llama, I love to","[_Hello my name is Llama, I love to LLama_\n ..."
